# Algorithm selection

In this script, we train many "quick and dirty" classifiers with default parameters and choose the best one for further optimization.

To run this script, the following data files are needed:
- dt_final.xlsx (attached)

In [1]:
# data
import pandas as pd
import numpy as np
import csv

# misc
import os
import re
import time
import ast
import warnings
import math
import copy
import matplotlib.pyplot as plt

# ML
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn import metrics
import scikitplot as skplt

from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from tensorflow.keras import layers

In [2]:
os.chdir('/Users/ladarudnitckaia/Desktop/Master Thesis/9. ML')
data = pd.read_excel('dt_final.xlsx')

## 1 Data preparation

In [3]:
dt = data[[
 'label4',
 'glove_vec300_norm', # normalize
 'tfidf_art', # normalize
 'negative_conc',
 'positive_conc',
 'weak_subj',
 'strong_subj',
 'hyperbolic_terms',
 'attitude_markers',
 'kill_verbs',
 'bias_lexicon',
 'assertive_verbs',
 'factive_verbs',
 'report_verbs',
 'implicative_verbs',
 'hedges',
 'boosters',
 'affect ',
 'posemo ',
 'negemo ',
 'anx ',
 'anger ',
 'sad ',
 'social ',
 'family ',
 'friend ',
 'female ',
 'male ',
 'cogproc ',
 'insight ',
 'cause ',
 'discrep ',
 'tentat ',
 'certain ',
 'differ ',
 'percept ',
 'see ',
 'hear ',
 'feel ',
 'bio ',
 'body ',
 'health ',
 'sexual ',
 'ingest ',
 'drives ',
 'affiliation ',
 'achieve ',
 'power ',
 'reward ',
 'risk ',
 'focuspast ',
 'focuspresent ',
 'focusfuture ',
 'relativ ',
 'motion ',
 'space ',
 'time ',
 'work ',
 'leisure ',
 'home ',
 'money ',
 'relig ',
 'death ',
 'informal ',
 'swear ',
 'netspeak ',
 'assent ',
 'nonflu ',
 'filler ',
 'pos_ADJ',
 'pos_ADP',
 'pos_ADV',
 'pos_AUX',
 'pos_DET',
 'pos_INTJ',
 'pos_NOUN',
 'pos_PRON',
 'pos_PROPN',
 'pos_SCONJ',
 'pos_VERB',
 'pos_X',
 'dep_ROOT',
 'dep_acl',
 'dep_acomp',
 'dep_advcl',
 'dep_advmod',
 'dep_agent',
 'dep_amod',
 'dep_appos',
 'dep_attr',
 'dep_aux',
 'dep_auxpass',
 'dep_case',
 'dep_cc',
 'dep_ccomp',
 'dep_compound',
 'dep_conj',
 'dep_csubj',
 'dep_dative',
 'dep_dep',
 'dep_det',
 'dep_dobj',
 'dep_expl',
 'dep_intj',
 'dep_mark',
 'dep_meta',
 'dep_neg',
 'dep_nmod',
 'dep_npadvmod',
 'dep_nsubj',
 'dep_nsubjpass',
 'dep_nummod',
 'dep_oprd',
 'dep_parataxis',
 'dep_pcomp',
 'dep_pobj',
 'dep_poss',
 'dep_preconj',
 'dep_predet',
 'dep_prep',
 'dep_prt',
 'dep_punct',
 'dep_quantmod',
 'dep_relcl',
 'dep_xcomp',
 'ne_CARDINAL',
 'ne_DATE',
 'ne_EVENT',
 'ne_FAC',
 'ne_GPE',
 'ne_LANGUAGE',
 'ne_LAW',
 'ne_LOC',
 'ne_MONEY',
 'ne_NORP',
 'ne_ORDINAL',
 'ne_ORG',
 'ne_PERCENT',
 'ne_PERSON',
 'ne_PRODUCT',
 'ne_QUANTITY',
 'ne_TIME',
 'ne_WORK_OF_ART',
 'negative_conc_context',
 'positive_conc_context',
 'weak_subj_context',
 'strong_subj_context',
 'hyperbolic_terms_context',
 'attitude_markers_context',
 'kill_verbs_context',
 'bias_lexicon_context',
 'assertive_verbs_context',
 'factive_verbs_context',
 'report_verbs_context',
 'implicative_verbs_context',
 'hedges_context',
 'boosters_context',
 'affect _context',
 'posemo _context',
 'negemo _context',
 'anx _context',
 'anger _context',
 'sad _context',
 'social _context',
 'family _context',
 'friend _context',
 'female _context',
 'male _context',
 'cogproc _context',
 'insight _context',
 'cause _context',
 'discrep _context',
 'tentat _context',
 'certain _context',
 'differ _context',
 'percept _context',
 'see _context',
 'hear _context',
 'feel _context',
 'bio _context',
 'body _context',
 'health _context',
 'sexual _context',
 'ingest _context',
 'drives _context',
 'affiliation _context',
 'achieve _context',
 'power _context',
 'reward _context',
 'risk _context',
 'focuspast _context',
 'focuspresent _context',
 'focusfuture _context',
 'relativ _context',
 'motion _context',
 'space _context',
 'time _context',
 'work _context',
 'leisure _context',
 'home _context',
 'money _context',
 'relig _context',
 'death _context',
 'informal _context',
 'swear _context',
 'netspeak _context',
 'assent _context',
 'nonflu _context',
 'filler _context',
 'pos_ADJ_context',
 'pos_ADP_context',
 'pos_ADV_context',
 'pos_AUX_context',
 'pos_DET_context',
 'pos_INTJ_context',
 'pos_NOUN_context',
 'pos_PRON_context',
 'pos_PROPN_context',
 'pos_SCONJ_context',
 'pos_VERB_context',
 'pos_X_context',
 'dep_ROOT_context',
 'dep_acl_context',
 'dep_acomp_context',
 'dep_advcl_context',
 'dep_advmod_context',
 'dep_agent_context',
 'dep_amod_context',
 'dep_appos_context',
 'dep_attr_context',
 'dep_aux_context',
 'dep_auxpass_context',
 'dep_case_context',
 'dep_cc_context',
 'dep_ccomp_context',
 'dep_compound_context',
 'dep_conj_context',
 'dep_csubj_context',
 'dep_dative_context',
 'dep_dep_context',
 'dep_det_context',
 'dep_dobj_context',
 'dep_expl_context',
 'dep_intj_context',
 'dep_mark_context',
 'dep_meta_context',
 'dep_neg_context',
 'dep_nmod_context',
 'dep_npadvmod_context',
 'dep_nsubj_context',
 'dep_nsubjpass_context',
 'dep_nummod_context',
 'dep_oprd_context',
 'dep_parataxis_context',
 'dep_pcomp_context',
 'dep_pobj_context',
 'dep_poss_context',
 'dep_preconj_context',
 'dep_predet_context',
 'dep_prep_context',
 'dep_prt_context',
 'dep_punct_context',
 'dep_quantmod_context',
 'dep_relcl_context',
 'dep_xcomp_context',
 'ne_CARDINAL_context',
 'ne_DATE_context',
 'ne_EVENT_context',
 'ne_FAC_context',
 'ne_GPE_context',
 'ne_LAW_context',
 'ne_LOC_context',
 'ne_MONEY_context',
 'ne_NORP_context',
 'ne_ORDINAL_context',
 'ne_ORG_context',
 'ne_PERCENT_context',
 'ne_PERSON_context',
 'ne_PRODUCT_context',
 'ne_QUANTITY_context',
 'ne_TIME_context',
 'ne_WORK_OF_ART_context',
 'ne_LANGUAGE_context']]

In [4]:
print('Number of missing values:')
mis = (dt.isnull().sum()).to_frame()
for i, row in mis.iterrows():
    if row[0] > 0:
        print(i, row[0])

# Delete the rows with missing article text:
dt = dt.dropna(subset=['tfidf_art'])

Number of missing values:
tfidf_art 658


In [5]:
x = dt.drop(['label4'], 1)
y4 = dt[['label4']]

In [6]:
x_scaled = copy.deepcopy(x)

scaler = MinMaxScaler(feature_range=(0, 1))
x_scaled['glove_vec300_norm'] = scaler.fit_transform(x_scaled[['glove_vec300_norm']])
x_scaled['tfidf_art'] = scaler.fit_transform(x_scaled[['tfidf_art']])

In [7]:
train_features, test_features, train_labels, test_labels = train_test_split(x, y4, 
                                                                            test_size = 0.20, random_state = 42)
train_features_sc, test_features_sc, train_labels, test_labels = train_test_split(x_scaled, y4, 
                                                                            test_size = 0.20, random_state = 42)

In [8]:
feature_list = list(x.columns)
len(feature_list)

282

In [9]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (24908, 282)
Training Labels Shape: (24908, 1)
Testing Features Shape: (6228, 282)
Testing Labels Shape: (6228, 1)


## 2 Select the algorithm among many classic classifiers

By default 1 is the positive class, 0 is the negative class
- High precision relates to a low false positive rate
- High recall relates to a low false negative rate.

In [10]:
sorted(metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

In [11]:
scoring = ['accuracy', 'f1', 'roc_auc', 'precision', 'recall']

In [12]:
print('Biased words:',round(len(y4[y4['label4']==1])/len(y4)*100,0),'%')
print('Biased words:',round(len(y4[y4['label4']==0])/len(y4)*100,0),'%')

Biased words: 10.0 %
Biased words: 90.0 %


### 2.1 Logistic regression

In [245]:
logreg_w = LogisticRegressionCV(random_state=42, class_weight = {0:10, 1:90})
scores_logreg_w = cross_validate(logreg_w, x_scaled, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to c

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to c

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/Users/

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_

In [246]:
print("F1: %0.2f (+/- %0.2f)" % (scores_logreg_w['test_f1'].mean(), scores_logreg_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_logreg_w['test_precision'].mean(), scores_logreg_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_logreg_w['test_recall'].mean(), scores_logreg_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_logreg_w['test_roc_auc'].mean(), scores_logreg_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_logreg_w['test_accuracy'].mean(), scores_logreg_w['test_accuracy'].std() * 2))

F1: 0.38 (+/- 0.05)
Precision: 0.26 (+/- 0.05)
Recall: 0.67 (+/- 0.06)
AUC: 0.82 (+/- 0.03)
Accuracy: 0.79 (+/- 0.05)


In [268]:
pred_logreg_w_cm = logreg_w.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_logreg_w_cm = pred_logreg_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_logreg_w_cm).transpose())

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to c

Confusion matrix:
 [[4510  194]
 [1102  422]]


### 2.2 LDA

In [23]:
lda = LinearDiscriminantAnalysis()
scores_lda = cross_validate(lda, x_scaled, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y 

In [24]:
print("F1: %0.2f (+/- %0.2f)" % (scores_lda['test_f1'].mean(), scores_lda['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_lda['test_precision'].mean(), scores_lda['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_lda['test_recall'].mean(), scores_lda['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_lda['test_roc_auc'].mean(), scores_lda['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_lda['test_accuracy'].mean(), scores_lda['test_accuracy'].std() * 2))

F1: 0.41 (+/- 0.04)
Precision: 0.50 (+/- 0.08)
Recall: 0.35 (+/- 0.06)
AUC: 0.82 (+/- 0.04)
Accuracy: 0.90 (+/- 0.01)


In [25]:
pred_lda_cm = lda.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_lda_cm = pred_lda_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_lda_cm).transpose())

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[5408  408]
 [ 204  208]]


### 2.3 QDA

In [28]:
qda = QuadraticDiscriminantAnalysis()
scores_qda = cross_validate(qda, x_scaled, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-pack

In [29]:
print("F1: %0.2f (+/- %0.2f)" % (scores_qda['test_f1'].mean(), scores_qda['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_qda['test_precision'].mean(), scores_qda['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_qda['test_recall'].mean(), scores_qda['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_qda['test_roc_auc'].mean(), scores_qda['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_qda['test_accuracy'].mean(), scores_qda['test_accuracy'].std() * 2))

F1: 0.19 (+/- 0.00)
Precision: 0.10 (+/- 0.00)
Recall: 0.99 (+/- 0.02)
AUC: 0.76 (+/- 0.03)
Accuracy: 0.18 (+/- 0.02)


In [30]:
pred_qda_cm = qda.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_qda_cm = pred_qda_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_qda_cm).transpose())

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[ 562    7]
 [5050  609]]


/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


### 2.4 Naive Bayes

In [33]:
nb = GaussianNB()
scores_nb = cross_validate(nb, x_scaled, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d

In [34]:
print("F1: %0.2f (+/- %0.2f)" % (scores_nb['test_f1'].mean(), scores_nb['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_nb['test_precision'].mean(), scores_nb['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_nb['test_recall'].mean(), scores_nb['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_nb['test_roc_auc'].mean(), scores_nb['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_nb['test_accuracy'].mean(), scores_nb['test_accuracy'].std() * 2))

F1: 0.20 (+/- 0.01)
Precision: 0.11 (+/- 0.00)
Recall: 0.98 (+/- 0.03)
AUC: 0.59 (+/- 0.03)
Accuracy: 0.24 (+/- 0.03)


In [35]:
pred_nb_cm = nb.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_nb_cm = pred_nb_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_nb_cm).transpose())

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[ 893   12]
 [4719  604]]


In [39]:
nb_imb = ComplementNB()
scores_nb_imb = cross_validate(nb_imb, x_scaled, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y 

In [40]:
print("F1: %0.2f (+/- %0.2f)" % (scores_nb_imb['test_f1'].mean(), scores_nb_imb['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_nb_imb['test_precision'].mean(), scores_nb_imb['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_nb_imb['test_recall'].mean(), scores_nb_imb['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_nb_imb['test_roc_auc'].mean(), scores_nb_imb['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_nb_imb['test_accuracy'].mean(), scores_nb_imb['test_accuracy'].std() * 2))

F1: 0.35 (+/- 0.04)
Precision: 0.23 (+/- 0.03)
Recall: 0.74 (+/- 0.05)
AUC: 0.82 (+/- 0.03)
Accuracy: 0.74 (+/- 0.05)


In [41]:
pred_nb_imb_cm = nb_imb.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_nb_imb_cm = pred_nb_imb_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_nb_imb_cm).transpose())

Confusion matrix:
 [[4190  157]
 [1422  459]]


/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### 2.5 KNN

In [43]:
knn = KNeighborsClassifier()
scores_knn = cross_validate(knn, x_scaled, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packa

In [44]:
print("F1: %0.2f (+/- %0.2f)" % (scores_knn['test_f1'].mean(), scores_knn['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_knn['test_precision'].mean(), scores_knn['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_knn['test_recall'].mean(), scores_knn['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_knn['test_roc_auc'].mean(), scores_knn['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_knn['test_accuracy'].mean(), scores_knn['test_accuracy'].std() * 2))

F1: 0.21 (+/- 0.04)
Precision: 0.45 (+/- 0.09)
Recall: 0.14 (+/- 0.03)
AUC: 0.70 (+/- 0.03)
Accuracy: 0.90 (+/- 0.01)


In [45]:
pred_knn_cm = knn.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_knn_cm = pred_knn_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_knn_cm).transpose())

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


Confusion matrix:
 [[5526  526]
 [  86   90]]


### 2.6 Decision Tree

In [286]:
dt = DecisionTreeClassifier(random_state=42)
scores_dt = cross_validate(dt, x, y4, cv=10, scoring=scoring)

In [287]:
print("F1: %0.2f (+/- %0.2f)" % (scores_dt['test_f1'].mean(), scores_dt['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_dt['test_precision'].mean(), scores_dt['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_dt['test_recall'].mean(), scores_dt['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_dt['test_roc_auc'].mean(), scores_dt['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_dt['test_accuracy'].mean(), scores_dt['test_accuracy'].std() * 2))

F1: 0.31 (+/- 0.05)
Precision: 0.31 (+/- 0.05)
Recall: 0.32 (+/- 0.06)
AUC: 0.62 (+/- 0.03)
Accuracy: 0.87 (+/- 0.01)


In [288]:
pred_dt_cm = dt.fit(train_features, train_labels).predict(test_features)
pred_binary_dt_cm = pred_dt_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_dt_cm).transpose())

Confusion matrix:
 [[5158  404]
 [ 454  212]]


In [289]:
dt_w = DecisionTreeClassifier(random_state=42, class_weight = {0:10, 1:90})
scores_dt_w = cross_validate(dt_w, x, y4, cv=10, scoring=scoring)

In [290]:
print("F1: %0.2f (+/- %0.2f)" % (scores_dt_w['test_f1'].mean(), scores_dt_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_dt_w['test_precision'].mean(), scores_dt_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_dt_w['test_recall'].mean(), scores_dt_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_dt_w['test_roc_auc'].mean(), scores_dt_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_dt_w['test_accuracy'].mean(), scores_dt_w['test_accuracy'].std() * 2))

F1: 0.31 (+/- 0.04)
Precision: 0.30 (+/- 0.05)
Recall: 0.33 (+/- 0.05)
AUC: 0.62 (+/- 0.03)
Accuracy: 0.86 (+/- 0.02)


In [291]:
pred_dt_w_cm = dt_w.fit(train_features, train_labels).predict(test_features)
pred_binary_dt_w_cm = pred_dt_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_dt_w_cm).transpose())

Confusion matrix:
 [[5109  409]
 [ 503  207]]


### 2.7 Random Forest

In [329]:
rf = RandomForestClassifier(random_state=42, n_estimators=100)
scores_rf = cross_validate(rf, x, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please c

In [330]:
print("F1: %0.2f (+/- %0.2f)" % (scores_rf['test_f1'].mean(), scores_rf['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_rf['test_precision'].mean(), scores_rf['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_rf['test_recall'].mean(), scores_rf['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_rf['test_roc_auc'].mean(), scores_rf['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rf['test_accuracy'].mean(), scores_rf['test_accuracy'].std() * 2))

F1: 0.27 (+/- 0.05)
Precision: 0.72 (+/- 0.12)
Recall: 0.17 (+/- 0.04)
AUC: 0.83 (+/- 0.04)
Accuracy: 0.91 (+/- 0.00)


In [331]:
pred_rf_cm = rf.fit(train_features, train_labels).predict(test_features)
pred_binary_rf_cm = pred_rf_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_rf_cm).transpose())

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


Confusion matrix:
 [[5573  517]
 [  39   99]]


In [332]:
rf_w = RandomForestClassifier(random_state=42, class_weight = {0:10, 1:90}, n_estimators=100)
scores_rf_w = cross_validate(rf_w, x, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please c

In [333]:
print("F1: %0.2f (+/- %0.2f)" % (scores_rf_w['test_f1'].mean(), scores_rf_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_rf_w['test_precision'].mean(), scores_rf_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_rf_w['test_recall'].mean(), scores_rf_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_rf_w['test_roc_auc'].mean(), scores_rf_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rf_w['test_accuracy'].mean(), scores_rf_w['test_accuracy'].std() * 2))

F1: 0.26 (+/- 0.04)
Precision: 0.71 (+/- 0.12)
Recall: 0.16 (+/- 0.04)
AUC: 0.84 (+/- 0.03)
Accuracy: 0.91 (+/- 0.00)


In [334]:
pred_rf_w_cm = rf_w.fit(train_features, train_labels).predict(test_features)
pred_binary_rf_w_cm = pred_rf_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_rf_w_cm).transpose())

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


Confusion matrix:
 [[5573  514]
 [  39  102]]


### 2.8 SVM (linear kernel)

In [337]:
svm = SVC(kernel='linear', random_state=42)
scores_svm = cross_validate(svm, x_scaled, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

In [338]:
print("F1: %0.2f (+/- %0.2f)" % (scores_svm['test_f1'].mean(), scores_svm['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_svm['test_precision'].mean(), scores_svm['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_svm['test_recall'].mean(), scores_svm['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_svm['test_roc_auc'].mean(), scores_svm['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_svm['test_accuracy'].mean(), scores_svm['test_accuracy'].std() * 2))

F1: 0.19 (+/- 0.08)
Precision: 0.59 (+/- 0.15)
Recall: 0.11 (+/- 0.06)
AUC: 0.69 (+/- 0.06)
Accuracy: 0.91 (+/- 0.00)


In [339]:
pred_svm_cm = svm.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_svm_cm = pred_svm_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_svm_cm).transpose())

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


Confusion matrix:
 [[5564  551]
 [  48   65]]


In [19]:
start_time = time.time()
svm_w = SVC(kernel='linear', random_state=42, class_weight = {0:10, 1:90})
scores_svm_w = cross_validate(svm_w, x_scaled, y4, cv=5, scoring=scoring)
end_time = time.time()
print('Time to train weighted linear SVM on all features, 10-fold CV:', round((end_time-start_time),2), 'seconds')

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y 

Time to train weighted linear SVM on all features, 10-fold CV: 24258.84 seconds


In [20]:
print("F1: %0.2f (+/- %0.2f)" % (scores_svm_w['test_f1'].mean(), scores_svm_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_svm_w['test_precision'].mean(), scores_svm_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_svm_w['test_recall'].mean(), scores_svm_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_svm_w['test_roc_auc'].mean(), scores_svm_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_svm_w['test_accuracy'].mean(), scores_svm_w['test_accuracy'].std() * 2))

F1: 0.38 (+/- 0.04)
Precision: 0.26 (+/- 0.04)
Recall: 0.70 (+/- 0.06)
AUC: 0.83 (+/- 0.02)
Accuracy: 0.78 (+/- 0.05)


In [21]:
start_time = time.time()
pred_svm_w_cm = svm_w.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_svm_w_cm = pred_svm_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_svm_w_cm).transpose())
end_time = time.time()
print('Time to train weighted linear SVM on all features, 10-fold CV:', round((end_time-start_time),2), 'seconds')

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[4419  178]
 [1193  438]]
Time to train weighted linear SVM on all features, 10-fold CV: 4769.49 seconds


### 2.9 SVM (non-linear kernel)

In [ ]:
start_time = time.time()
svm_w = SVC(random_state=42, class_weight = {0:10, 1:90})
scores_svm_w = cross_validate(svm_w, x_scaled, y4, cv=5, scoring=scoring)
end_time = time.time()
print('Time to train weighted linear SVM on all features, 10-fold CV:', round((end_time-start_time),2), 'seconds')

In [191]:
print("F1: %0.2f (+/- %0.2f)" % (scores_svm_w['test_f1'].mean(), scores_svm_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_svm_w['test_precision'].mean(), scores_svm_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_svm_w['test_recall'].mean(), scores_svm_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_svm_w['test_roc_auc'].mean(), scores_svm_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_svm_w['test_accuracy'].mean(), scores_svm_w['test_accuracy'].std() * 2))

F1: 0.35 (+/- 0.04)
Precision: 0.39 (+/- 0.06)
Recall: 0.31 (+/- 0.05)
AUC: 0.78 (+/- 0.02)
Accuracy: 0.89 (+/- 0.01)


In [192]:
start_time = time.time()
pred_svm_w_cm = svm_w.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_svm_w_cm = pred_svm_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_svm_w_cm).transpose())
end_time = time.time()
print('Time to train weighted linear SVM on all features, 10-fold CV:', round((end_time-start_time),2), 'seconds')

Confusion matrix:
 [[5335  415]
 [ 277  201]]


### 2.10 XGBoost

In [49]:
xgb = xgboost.XGBClassifier(objective="binary:logistic", random_state=42)
scores_xgb = cross_validate(xgb, x, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A 

In [50]:
print("F1: %0.2f (+/- %0.2f)" % (scores_xgb['test_f1'].mean(), scores_xgb['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_xgb['test_precision'].mean(), scores_xgb['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_xgb['test_recall'].mean(), scores_xgb['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_xgb['test_roc_auc'].mean(), scores_xgb['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_xgb['test_accuracy'].mean(), scores_xgb['test_accuracy'].std() * 2))

F1: 0.35 (+/- 0.04)
Precision: 0.62 (+/- 0.09)
Recall: 0.25 (+/- 0.05)
AUC: 0.85 (+/- 0.03)
Accuracy: 0.91 (+/- 0.01)


In [51]:
pred_xgb_cm = xgb.fit(train_features, train_labels).predict(test_features)
pred_binary_xgb_cm = pred_xgb_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_xgb_cm).transpose())

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[5507  459]
 [ 105  157]]


In [56]:
scale_pos_weight = len(y4[y4['label4']==0])/len(y4[y4['label4']==1])
scale_pos_weight

9.455339153794492

In [57]:
xgb_w = xgboost.XGBClassifier(objective="binary:logistic", scale_pos_weight=scale_pos_weight, random_state=42)
scores_xgb_w = cross_validate(xgb_w, x, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A 

In [58]:
print("F1: %0.2f (+/- %0.2f)" % (scores_xgb_w['test_f1'].mean(), scores_xgb_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_xgb_w['test_precision'].mean(), scores_xgb_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_xgb_w['test_recall'].mean(), scores_xgb_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_xgb_w['test_roc_auc'].mean(), scores_xgb_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_xgb_w['test_accuracy'].mean(), scores_xgb_w['test_accuracy'].std() * 2))

F1: 0.42 (+/- 0.04)
Precision: 0.32 (+/- 0.04)
Recall: 0.64 (+/- 0.07)
AUC: 0.84 (+/- 0.03)
Accuracy: 0.83 (+/- 0.03)


In [59]:
pred_xgb_w_cm = xgb_w.fit(train_features, train_labels).predict(test_features)
pred_binary_xgb_w_cm = pred_xgb_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_xgb_w_cm).transpose())

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[4774  207]
 [ 838  409]]


### 2.11 MLP

In [61]:
mlp = MLPClassifier(random_state=42)
scores_mlp = cross_validate(mlp, x_scaled, y4, cv=10, scoring=scoring)

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptr

In [62]:
print("F1: %0.2f (+/- %0.2f)" % (scores_mlp['test_f1'].mean(), scores_mlp['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_mlp['test_precision'].mean(), scores_mlp['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_mlp['test_recall'].mean(), scores_mlp['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_mlp['test_roc_auc'].mean(), scores_mlp['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_mlp['test_accuracy'].mean(), scores_mlp['test_accuracy'].std() * 2))

F1: 0.32 (+/- 0.03)
Precision: 0.35 (+/- 0.06)
Recall: 0.30 (+/- 0.06)
AUC: 0.77 (+/- 0.03)
Accuracy: 0.88 (+/- 0.02)


In [63]:
pred_mlp_cm = mlp.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_mlp_cm = pred_mlp_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_mlp_cm).transpose())

/Users/ladarudnitckaia/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[5340  446]
 [ 272  170]]


In [169]:
def define_model(n_input):
    # define model
    mlp_w = Sequential()
    # define first hidden layer and visible layer
    mlp_w.add(layers.Dense(100, input_dim=n_input, activation='relu'))
    # define output layer
    mlp_w.add(layers.Dense(1, activation='sigmoid'))
    # define loss and optimizer
    mlp_w.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC','Precision','Recall'])
    return mlp_w

In [179]:
# K-fold Cross Validation model evaluation
start_time = time.time()

num_folds = 10
fold_no = 1
F1_per_fold = []
precision_per_fold = []
recall_per_fold = []
AUC_per_fold = []

kf = KFold(n_splits=10, random_state=42, shuffle=True)

x_scaled_np = x_scaled.to_numpy()
y4_np = y4.to_numpy()

for train_index, test_index in kf.split(x_scaled_np):
    X_train, X_test = x_scaled_np[train_index], x_scaled_np[test_index]
    y_train, y_test = y4_np[train_index], y4_np[test_index]

    # Generate a print
    print(f'Training for fold {fold_no} ...')
    
    mlp_w = define_model(len(feature_list))

    # Fit data to model
    mlp_w.fit(X_train, y_train, epochs=200, class_weight={0:10, 1:90})
    mlp_w_pred = mlp_w.predict(X_test)
    mlp_w_pred_bin = mlp_w_pred.round()
    
    F1_per_fold.append(metrics.f1_score(y_test, mlp_w_pred_bin, pos_label=1))
    precision_per_fold.append(metrics.precision_score(y_test, mlp_w_pred_bin, pos_label=1))
    recall_per_fold.append(metrics.recall_score(y_test, mlp_w_pred_bin, pos_label=1))
    AUC_per_fold.append(metrics.roc_auc_score(y_test, mlp_w_pred_bin))
    print('CF:', metrics.confusion_matrix(y_test, mlp_w_pred_bin).transpose())

    # Increase fold number
    fold_no = fold_no + 1

end_time = time.time()
print('Time to train weighted MLP on all features, 10-fold CV:', round((end_time-start_time),2), 'seconds')

Training for fold 1 ...
Epoch 1/200
876/876 [==============================] - 1s 2ms/step - loss: 9.2116 - auc: 0.8150 - precision: 0.2317 - recall: 0.7134
Epoch 2/200
876/876 [==============================] - 1s 2ms/step - loss: 8.3817 - auc: 0.8512 - precision: 0.2601 - recall: 0.7601
Epoch 3/200
876/876 [==============================] - 2s 2ms/step - loss: 8.0130 - auc: 0.8659 - precision: 0.2680 - recall: 0.7836
Epoch 4/200
876/876 [==============================] - 2s 2ms/step - loss: 7.6466 - auc: 0.8793 - precision: 0.2817 - recall: 0.8034
Epoch 5/200
876/876 [==============================] - 2s 2ms/step - loss: 7.2288 - auc: 0.8938 - precision: 0.3005 - recall: 0.8277
Epoch 6/200
876/876 [==============================] - 2s 2ms/step - loss: 6.7917 - auc: 0.9074 - precision: 0.3193 - recall: 0.8445: 0s - loss: 6.7613 - auc: 0.9086 - precision: 0.3211 - recall: 0.
Epoch 7/200
876/876 [==============================] - 2s 2ms/step - loss: 6.3664 - auc: 0.9204 - precision: 0.3

876/876 [==============================] - 2s 2ms/step - loss: 0.1100 - auc: 0.9998 - precision: 0.9856 - recall: 0.9985
Epoch 117/200
876/876 [==============================] - 2s 2ms/step - loss: 0.1016 - auc: 0.9998 - precision: 0.9835 - recall: 0.9996
Epoch 118/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1285 - auc: 0.9998 - precision: 0.9788 - recall: 0.9989
Epoch 119/200
876/876 [==============================] - 2s 2ms/step - loss: 0.1061 - auc: 0.9998 - precision: 0.9817 - recall: 0.9996
Epoch 120/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1219 - auc: 0.9998 - precision: 0.9852 - recall: 0.9981
Epoch 121/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1166 - auc: 0.9999 - precision: 0.9838 - recall: 0.9993
Epoch 122/200
876/876 [==============================] - 2s 2ms/step - loss: 0.0998 - auc: 0.9998 - precision: 0.9831 - recall: 0.9989
Epoch 123/200
876/876 [==============================] - 2s 2ms/step 

876/876 [==============================] - 2s 2ms/step - loss: 1.5557 - auc: 0.9950 - precision: 0.7226 - recall: 0.9914: 0s - loss: 1.3975 - auc: 0.9963 - 
Epoch 30/200
876/876 [==============================] - 2s 2ms/step - loss: 1.4813 - auc: 0.9955 - precision: 0.7332 - recall: 0.9944: 0s - loss: 1.4804 - auc: 0.9955 - precision: 0.7342 - recall: 0.
Epoch 31/200
876/876 [==============================] - 1s 2ms/step - loss: 1.4180 - auc: 0.9958 - precision: 0.7476 - recall: 0.9906
Epoch 32/200
876/876 [==============================] - 2s 2ms/step - loss: 1.3059 - auc: 0.9966 - precision: 0.7594 - recall: 0.9951
Epoch 33/200
876/876 [==============================] - 2s 2ms/step - loss: 1.2666 - auc: 0.9968 - precision: 0.7672 - recall: 0.9921
Epoch 34/200
876/876 [==============================] - 2s 2ms/step - loss: 1.1728 - auc: 0.9972 - precision: 0.7860 - recall: 0.9940
Epoch 35/200
876/876 [==============================] - 2s 2ms/step - loss: 1.1303 - auc: 0.9975 - precisio

876/876 [==============================] - 2s 2ms/step - loss: 0.1559 - auc: 0.9996 - precision: 0.9761 - recall: 0.9977
Epoch 146/200
876/876 [==============================] - 2s 2ms/step - loss: 0.1384 - auc: 0.9998 - precision: 0.9772 - recall: 0.9992
Epoch 147/200
876/876 [==============================] - 2s 2ms/step - loss: 0.1007 - auc: 0.9998 - precision: 0.9856 - recall: 0.9992
Epoch 148/200
876/876 [==============================] - 2s 2ms/step - loss: 0.0866 - auc: 0.9998 - precision: 0.9881 - recall: 0.9989
Epoch 149/200
876/876 [==============================] - 1s 2ms/step - loss: 0.0834 - auc: 0.9998 - precision: 0.9885 - recall: 0.9992
Epoch 150/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1366 - auc: 0.9995 - precision: 0.9874 - recall: 0.9989
Epoch 151/200
876/876 [==============================] - 2s 2ms/step - loss: 0.1183 - auc: 0.9998 - precision: 0.9848 - recall: 0.9989
Epoch 152/200
876/876 [==============================] - 1s 2ms/step 

876/876 [==============================] - 1s 2ms/step - loss: 0.2314 - auc: 0.9998 - precision: 0.9662 - recall: 0.9989
Epoch 63/200
876/876 [==============================] - 1s 2ms/step - loss: 0.2466 - auc: 0.9998 - precision: 0.9606 - recall: 0.9985
Epoch 64/200
876/876 [==============================] - 1s 2ms/step - loss: 0.2217 - auc: 0.9998 - precision: 0.9704 - recall: 0.9989
Epoch 65/200
876/876 [==============================] - 1s 2ms/step - loss: 0.2028 - auc: 0.9999 - precision: 0.9690 - recall: 0.9989
Epoch 66/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1952 - auc: 0.9999 - precision: 0.9728 - recall: 0.9989
Epoch 67/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1756 - auc: 0.9999 - precision: 0.9767 - recall: 0.9996
Epoch 68/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1770 - auc: 0.9998 - precision: 0.9729 - recall: 0.9996
Epoch 69/200
876/876 [==============================] - 1s 2ms/step - loss:

...

876/876 [==============================] - 1s 2ms/step - loss: 0.2370 - auc: 0.9998 - precision: 0.9693 - recall: 0.9985
Epoch 71/200
876/876 [==============================] - 1s 2ms/step - loss: 0.2265 - auc: 0.9998 - precision: 0.9632 - recall: 0.9981
Epoch 72/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1846 - auc: 0.9998 - precision: 0.9704 - recall: 0.9996
Epoch 73/200
876/876 [==============================] - 2s 2ms/step - loss: 0.1847 - auc: 0.9999 - precision: 0.9728 - recall: 0.9989
Epoch 74/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1844 - auc: 0.9998 - precision: 0.9721 - recall: 0.9989
Epoch 75/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1907 - auc: 0.9998 - precision: 0.9689 - recall: 0.9992
Epoch 76/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1843 - auc: 0.9999 - precision: 0.9682 - recall: 0.9985
Epoch 77/200
876/876 [==============================] - 1s 2ms/step - loss:

876/876 [==============================] - 2s 2ms/step - loss: 0.0845 - auc: 0.9999 - precision: 0.9855 - recall: 0.9992
Epoch 190/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1243 - auc: 0.9996 - precision: 0.9888 - recall: 0.9985
Epoch 191/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1093 - auc: 0.9998 - precision: 0.9851 - recall: 0.9981
Epoch 192/200
876/876 [==============================] - 2s 2ms/step - loss: 0.0842 - auc: 0.9998 - precision: 0.9895 - recall: 0.9989
Epoch 193/200
876/876 [==============================] - 2s 2ms/step - loss: 0.1165 - auc: 0.9998 - precision: 0.9840 - recall: 0.9989
Epoch 194/200
876/876 [==============================] - 2s 2ms/step - loss: 0.0994 - auc: 0.9997 - precision: 0.9903 - recall: 0.9989: 1s - loss: 0.0247 - auc: 0.9999 - precision: 0.9977 - recall: 1. - ETA: 1s - loss: 0.1041 - au
Epoch 195/200
876/876 [==============================] - 1s 2ms/step - loss: 0.0611 - auc: 0.9998 - precisio

876/876 [==============================] - 2s 2ms/step - loss: 0.1183 - auc: 0.9999 - precision: 0.9839 - recall: 0.9985
Epoch 109/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1040 - auc: 0.9998 - precision: 0.9857 - recall: 0.9989
Epoch 110/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1209 - auc: 0.9998 - precision: 0.9810 - recall: 0.9981
Epoch 111/200
876/876 [==============================] - 1s 2ms/step - loss: 0.0923 - auc: 0.9998 - precision: 0.9882 - recall: 0.9996
Epoch 112/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1000 - auc: 0.9999 - precision: 0.9835 - recall: 0.9981
Epoch 113/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1048 - auc: 0.9999 - precision: 0.9828 - recall: 0.9989
Epoch 114/200
876/876 [==============================] - 1s 2ms/step - loss: 0.0743 - auc: 0.9999 - precision: 0.9897 - recall: 1.0000
Epoch 115/200
876/876 [==============================] - 1s 2ms/step 

876/876 [==============================] - 1s 2ms/step - loss: 1.6535 - auc: 0.9944 - precision: 0.7070 - recall: 0.9933: 1s - loss: 1.4441 - 
Epoch 28/200
876/876 [==============================] - 1s 2ms/step - loss: 1.5747 - auc: 0.9950 - precision: 0.7192 - recall: 0.9915
Epoch 29/200
876/876 [==============================] - 1s 2ms/step - loss: 1.4907 - auc: 0.9955 - precision: 0.7334 - recall: 0.9944
Epoch 30/200
876/876 [==============================] - 1s 2ms/step - loss: 1.4094 - auc: 0.9958 - precision: 0.7494 - recall: 0.9944
Epoch 31/200
876/876 [==============================] - 1s 2ms/step - loss: 1.2919 - auc: 0.9966 - precision: 0.7627 - recall: 0.9944
Epoch 32/200
876/876 [==============================] - 1s 2ms/step - loss: 1.2308 - auc: 0.9969 - precision: 0.7712 - recall: 0.9955
Epoch 33/200
876/876 [==============================] - 1s 2ms/step - loss: 1.1671 - auc: 0.9973 - precision: 0.7773 - recall: 0.9955
Epoch 34/200
876/876 [==============================]

876/876 [==============================] - 2s 2ms/step - loss: 0.1027 - auc: 0.9997 - precision: 0.9904 - recall: 0.9993
Epoch 148/200
876/876 [==============================] - 2s 2ms/step - loss: 0.0737 - auc: 0.9999 - precision: 0.9930 - recall: 0.9996
Epoch 149/200
876/876 [==============================] - 2s 2ms/step - loss: 0.0745 - auc: 0.9999 - precision: 0.9908 - recall: 0.9993
Epoch 150/200
876/876 [==============================] - 1s 2ms/step - loss: 0.1092 - auc: 0.9998 - precision: 0.9875 - recall: 0.9981
Epoch 151/200
876/876 [==============================] - 2s 2ms/step - loss: 0.1103 - auc: 0.9998 - precision: 0.9821 - recall: 0.9993
Epoch 152/200
876/876 [==============================] - 2s 2ms/step - loss: 0.1227 - auc: 0.9996 - precision: 0.9864 - recall: 0.9985
Epoch 153/200
876/876 [==============================] - 1s 2ms/step - loss: 0.0881 - auc: 0.9999 - precision: 0.9897 - recall: 0.9985
Epoch 154/200
876/876 [==============================] - 1s 2ms/step 

In [184]:
print("F1: %0.2f (+/- %0.2f)" % (statistics.mean(F1_per_fold), statistics.pstdev(F1_per_fold) * 2))
print("Precision: %0.2f (+/- %0.2f)" % (statistics.mean(precision_per_fold), statistics.pstdev(precision_per_fold) * 2))
print("Recall: %0.2f (+/- %0.2f)" % (statistics.mean(recall_per_fold), statistics.pstdev(recall_per_fold) * 2))
print("AUC: %0.2f (+/- %0.2f)" % (statistics.mean(AUC_per_fold), statistics.pstdev(AUC_per_fold) * 2))

F1: 0.34 (+/- 0.06)
Precision: 0.35 (+/- 0.07)
Recall: 0.33 (+/- 0.06)
AUC: 0.63 (+/- 0.03)
